In [38]:
import pandas as pd
import numpy as np
from collections import Counter, defaultdict
from IPython.core.debugger import set_trace
import random

import bcubed # https://github.com/hhromic/python-bcubed

# Evaluate cluster assignments with simple Taylor accuracy

In [17]:
# Load clustering assignments
# clusters_csvpath = 'ao3_fanfiction_kmeans_results_10kfics_10kvocab_no-stopwords_yes-names.csv'
clusters_csvpath = 'ao3_hp_kmeans_10kfics_10kvocab_svd1k_no-stopwords_no-names.csv'
clusters_data = pd.read_csv(clusters_csvpath)
print(len(clusters_data))
print(clusters_data.columns)

clusters_data.set_index('work_id', inplace=True)

10000
Index(['work_id', 'author_hash', 'category'], dtype='object')


In [18]:
# Load actual author assignments
actual_csvpath = '/usr2/mamille2/AO3Scraper/data/hp_ao3_metadata.csv'

actual_data = pd.read_csv(actual_csvpath)
print(len(actual_data))
print(actual_data.columns)
actual_data.set_index('work_id', inplace=True)

116622
Index(['Unnamed: 0', 'work_id', 'title', 'author_hash', 'rating', 'category',
       'fandom', 'relationship', 'character', 'additional tags', 'language',
       'published', 'status', 'status date', 'words', 'chapters', 'comments',
       'kudos', 'bookmarks', 'hits'],
      dtype='object')


In [9]:
actual_data.head()

,Unnamed: 0,title,author_hash,rating,category,fandom,relationship,character,additional tags,language,published,status,status date,words,chapters,comments,kudos,bookmarks,hits
work_id,,,,,,,,,,,,,,,,,,,
234222,0,Then Comes a Mist and a Weeping Rain,992fc6e21a8c9ab9788c92ff3bfb321d,Explicit,M/M,Harry Potter - J. K. Rowling,Draco Malfoy/Harry Potter,"Harry Potter, Draco Malfoy, Gregory Goyle, Mil...","Humor, Romance, Hogwarts Eighth Year, HP: EWE,...",English,2011-08-04,Completed,2011-08-04,21139.0,1/1,466.0,22078.0,4944.0,275581.0
234222,1,Then Comes a Mist and a Weeping Rain,992fc6e21a8c9ab9788c92ff3bfb321d,Explicit,M/M,Harry Potter - J. K. Rowling,Draco Malfoy/Harry Potter,"Harry Potter, Draco Malfoy, Gregory Goyle, Mil...","Humor, Romance, Hogwarts Eighth Year, HP: EWE,...",English,2011-08-04,Completed,2011-08-04,21139.0,1/1,466.0,22078.0,4944.0,275583.0
485218,2,Storm in a Teacup,992fc6e21a8c9ab9788c92ff3bfb321d,Mature,M/M,Harry Potter - J. K. Rowling,Draco Malfoy/Harry Potter,Draco Malfoy,"Hogwarts Eighth Year, Humor, Romance, Fluff",English,2012-08-13,Completed,2012-08-13,7954.0,1/1,257.0,15371.0,2760.0,157682.0
2734082,3,Tea and No Sympathy,dd924924ab5a4e01ba908e79b8fa897f,Explicit,M/M,Harry Potter - J. K. Rowling,Draco Malfoy/Harry Potter,"Narcissa Malfoy, Lucius Malfoy, Hermione Grang...","Slow Build, Draco-centric, Tea Drinking, Swear...",English,2014-12-23,Completed,2014-12-23,70045.0,11/11,853.0,14577.0,4074.0,202789.0
1439476,4,An Issue of Consequence,992fc6e21a8c9ab9788c92ff3bfb321d,Explicit,M/M,Harry Potter - J. K. Rowling,Draco Malfoy/Harry Potter,"Draco Malfoy, Harry Potter, Blaise Zabini, Pan...","HP: EWE, Hogwarts Eighth Year, Romance, Myster...",English,2014-04-09,Completed,2014-04-15,20798.0,5/5,397.0,14412.0,2615.0,217711.0


In [19]:
# Build estimated clusters
gped = clusters_data.groupby(['category'])
est_clusters = gped.groups
est_clusters

{0: Int64Index([6756529], dtype='int64', name='work_id'),
 1: Int64Index([  670656,   781685,  8164588,   180965, 13909653,  9861590,
             14709326,   461717,   390387,   839517,
             ...
              2802704, 13913991,  2368121,  4509612,  7868053, 10494276,
               745598, 12201360,  1908204,  8387122],
            dtype='int64', name='work_id', length=1054),
 2: Int64Index([8185430, 43511, 5082925], dtype='int64', name='work_id'),
 3: Int64Index([12315576], dtype='int64', name='work_id'),
 4: Int64Index([10851894, 8809930], dtype='int64', name='work_id'),
 5: Int64Index([14432568, 711139, 4846556], dtype='int64', name='work_id'),
 6: Int64Index([6376246, 11540361, 7506304], dtype='int64', name='work_id'),
 7: Int64Index([5163605, 4594071], dtype='int64', name='work_id'),
 8: Int64Index([1257940], dtype='int64', name='work_id'),
 9: Int64Index([871848, 375858, 597096], dtype='int64', name='work_id'),
 10: Int64Index([5941429, 71041, 75983], dtype='int64', name

In [23]:
# Build actual clusters (not sure if necessary)
# gped = actual_data.groupby(['author_hash'])
# actual_clusters = gped.groups
# actual_clusters

# actual_author = actual_data.loc[:, 'author_hash'].to_dict('index')
actual_author = actual_data['author_hash'].to_dict()
# actual_author = actual_data.loc[:, 'author_hash']
# actual_author.to_dict('index')
# print(type(actual_author))
len(actual_author)

10000

In [21]:
# Map clusters to actual authors (find actual author that yields most assignments, just greedily pick them)
remaining_authors = actual_data['author_hash'].unique().tolist()
est_authors = {}

# for i, cluster in list(est_clusters.items())[:1]:
for i, cluster in est_clusters.items():
    
    # Find best author
#     actual_assgns = [actual_author[wid] for wid in cluster]
    actual_assgns = [actual_author[wid] for wid in cluster if actual_author[wid] in remaining_authors]
#     print(actual_assgns)
    if len(actual_assgns) == 0:
        best_author = 'none'
    else:
        best_author = Counter(actual_assgns).most_common(1)[0][0]
        remaining_authors.remove(best_author)
    
    # Assign that authors to IDs
    for wid in cluster:
        est_authors[wid] = best_author
        
len(est_authors)

10000

In [22]:
# Compute accuracy of author assignments
correct = 0
for wid in est_authors:
     if est_authors[wid] == actual_author[wid]:
            correct += 1
            
print(correct/len(est_authors))

0.1554


# Evaluate cluster assignments with BCubed

In [10]:
# Load clustering assignments
# clusters_csvpath = 'ao3_fanfiction_kmeans_results_10kfics_10kvocab_no-stopwords_yes-names.csv'
clusters_csvpath = 'ao3_hp_kmeans_10kfics_10kvocab_svd1k_no-stopwords_no-names.csv'
clusters_data = pd.read_csv(clusters_csvpath)
print(len(clusters_data))
print(clusters_data.columns)

10000
Index(['work_id', 'author_hash', 'category'], dtype='object')


In [11]:
cluster_dict = {wid: set([c]) for wid, c in zip(clusters_data['work_id'], clusters_data['category'])}
len(cluster_dict)

10000

In [12]:
# Load actual author assignments
actual_csvpath = '/usr2/mamille2/AO3Scraper/data/hp_ao3_metadata.csv'

actual_data = pd.read_csv(actual_csvpath)
print(len(actual_data))
print(actual_data.columns)

116622
Index(['Unnamed: 0', 'work_id', 'title', 'author_hash', 'rating', 'category',
       'fandom', 'relationship', 'character', 'additional tags', 'language',
       'published', 'status', 'status date', 'words', 'chapters', 'comments',
       'kudos', 'bookmarks', 'hits'],
      dtype='object')


In [15]:
actual_dict = {}
for wid in clusters_data['work_id']:
    actual_dict[wid] = set([actual_data.loc[actual_data['work_id']==wid, 'author_hash'].tolist()[0]])
    
len(actual_dict)

10000

In [16]:
p = bcubed.precision(cluster_dict, actual_dict)
r = bcubed.recall(cluster_dict, actual_dict)
f1 = bcubed.fscore(p,r)
print(p)
print(r)
print(f1)

0.273270412739
0.262491777072
0.26777267088


# Evaluate random clustering

In [64]:
# Load actual author assignments
actual_csvpath = '/usr2/mamille2/AO3Scraper/data/hp_ao3_metadata.csv'

actual_data = pd.read_csv(actual_csvpath).drop_duplicates('work_id')
actual_data.set_index('work_id', inplace=True)
print(len(actual_data))
print(actual_data.columns)

116620
Index(['Unnamed: 0', 'title', 'author_hash', 'rating', 'category', 'fandom',
       'relationship', 'character', 'additional tags', 'language', 'published',
       'status', 'status date', 'words', 'chapters', 'comments', 'kudos',
       'bookmarks', 'hits'],
      dtype='object')


In [65]:
# Select stories that match clustering set-up
clusters_csvpath = 'ao3_hp_kmeans_10kfics_10kvocab_svd1k_no-stopwords_no-names.csv'
clusters_data = pd.read_csv(clusters_csvpath)
print(len(clusters_data))

actual_data = actual_data[actual_data.index.isin(clusters_data['work_id'])]
len(actual_data)

10000


10000

In [19]:
# Get stats from docs/cluster distro
cluster_sizes = actual_data['author_hash'].value_counts().tolist()
# mean_clusters = len(clusters_data['category'].unique())/len(cluster_dict)
# mean_clusters
print(sum(cluster_sizes))
print(np.mean(cluster_sizes))
print(np.std(cluster_sizes))

10000
5.46149645003
18.4484023025


In [14]:
unassigned = list(actual_data['work_id'].unique())
est_clusters = {}

for i, size in enumerate(cluster_sizes):
    samp = random.sample(unassigned, size)
    for wid in samp:
        est_clusters[wid] = str(i)
        unassigned.remove(wid)
        
len(est_clusters)

10000

In [36]:
cluster_sizes

[436,
 370,
 305,
 227,
 195,
 172,
 165,
 125,
 96,
 69,
 60,
 54,
 50,
 49,
 48,
 46,
 41,
 37,
 36,
 35,
 34,
 33,
 32,
 29,
 28,
 28,
 27,
 26,
 26,
 26,
 26,
 25,
 24,
 24,
 24,
 24,
 24,
 22,
 21,
 21,
 21,
 21,
 20,
 20,
 20,
 20,
 20,
 19,
 19,
 19,
 19,
 19,
 18,
 18,
 18,
 18,
 18,
 17,
 17,
 17,
 17,
 17,
 17,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8

In [37]:
print(list(est_clusters.items())[440:450])

[(6685732, '1'), (2451911, '1'), (14289726, '1'), (799462, '1'), (63118, '1'), (3177760, '1'), (10813851, '1'), (14375853, '1'), (2775542, '1'), (10905249, '1')]


In [16]:
# format for BCubed
est_clusters_bcubed = {wid: set(clu) for wid, clu in est_clusters.items()}
cluster_dict = est_clusters_bcubed

actual_dict = {wid: set(auth) for wid, auth in zip(actual_data['work_id'], actual_data['author_hash'])}

In [17]:
p = bcubed.precision(cluster_dict, actual_dict)
r = bcubed.recall(cluster_dict, actual_dict)
f1 = bcubed.fscore(p,r)
print(p)
print(r)
print(f1)

0.996027497958
0.0460770818118
0.0880795294467


In [21]:
# See dist of cluster sizes
cluster_size_ctr = Counter(cluster_sizes)
cluster_size_ctr.most_common()

[(2, 451),
 (1, 359),
 (3, 288),
 (4, 224),
 (5, 132),
 (6, 87),
 (7, 54),
 (8, 50),
 (10, 26),
 (9, 24),
 (15, 14),
 (12, 14),
 (11, 14),
 (14, 12),
 (13, 10),
 (16, 9),
 (17, 6),
 (24, 5),
 (20, 5),
 (19, 5),
 (18, 5),
 (26, 4),
 (21, 4),
 (28, 2),
 (436, 1),
 (370, 1),
 (305, 1),
 (227, 1),
 (195, 1),
 (172, 1),
 (165, 1),
 (125, 1),
 (96, 1),
 (69, 1),
 (60, 1),
 (54, 1),
 (50, 1),
 (49, 1),
 (48, 1),
 (46, 1),
 (41, 1),
 (37, 1),
 (36, 1),
 (35, 1),
 (34, 1),
 (33, 1),
 (32, 1),
 (29, 1),
 (27, 1),
 (25, 1),
 (22, 1)]

In [66]:
new_est_clusters = defaultdict(list)
for wid, clu in est_clusters.items():
    new_est_clusters[clu].append(wid)

In [69]:
# Build actual clusters (not sure if necessary)
# gped = actual_data.groupby(['author_hash'])
# actual_clusters = gped.groups
# actual_clusters

# actual_author = actual_data.loc[:, 'author_hash'].to_dict('index')
actual_author = actual_data['author_hash'].to_dict()
# actual_author = actual_data.loc[:, 'author_hash']
# actual_author.to_dict('index')
# print(type(actual_author))
len(actual_author)

10000

In [70]:
actual_author

{234222: '992fc6e21a8c9ab9788c92ff3bfb321d',
 485218: '992fc6e21a8c9ab9788c92ff3bfb321d',
 1439476: '992fc6e21a8c9ab9788c92ff3bfb321d',
 5744635: '2e753d482339ad1c21d067de1d1c0444',
 9065263: 'dd924924ab5a4e01ba908e79b8fa897f',
 1152329: '992fc6e21a8c9ab9788c92ff3bfb321d',
 987408: '9937e729280cfd44b03a0882b78cf844',
 4308786: '8b09a9a2800f63a0093ff8a9ff59bdde',
 717901: '1292774b9408d8dfc87f913dcbea3e40',
 3686949: '9499882807a11e318cdd4e7683108ac9',
 7176989: '2e753d482339ad1c21d067de1d1c0444',
 78824: '0a3ef4d88db4bc525b7cc10681f994fc',
 1092424: '6b1c9a0f43277ee331078f38670b0341',
 8169415: '29ec3e1432c70586e21ab42b2c6b9ed1',
 10171604: 'b67d5a81ebe22224ca8464d710a42be8',
 385354: '418780bd52e63495adb1a612657ef9ab',
 11075166: 'cf0c76530ffd6b916f5c76549ff4bf00',
 2469818: 'e2050633b44341954e4aa7390005a1b0',
 11221602: '073b4fe8eccde6aebdfb5dee32d9c08f',
 2276499: '8baa4c621ce8d63c9c3a6fc7658659d4',
 1110284: '992fc6e21a8c9ab9788c92ff3bfb321d',
 2469854: 'e2050633b44341954e4aa739000

In [71]:
# Map clusters to actual authors (find actual author that yields most assignments, just greedily pick them)
remaining_authors = actual_data['author_hash'].unique().tolist()
est_authors = {}

# for i, cluster in list(est_clusters.items())[:1]:
for i, cluster in new_est_clusters.items():
    
    # Find best author
#     actual_assgns = [actual_author[wid] for wid in cluster]
    actual_assgns = [actual_author[wid] for wid in cluster if actual_author[wid] in remaining_authors]
#     print(actual_assgns)
    if len(actual_assgns) == 0:
        best_author = 'none'
    else:
        best_author = Counter(actual_assgns).most_common(1)[0][0]
        remaining_authors.remove(best_author)
    
    # Assign that authors to IDs
    for wid in cluster:
        est_authors[wid] = best_author
        
len(est_authors)

10000

In [72]:
# Compute accuracy of author assignments
correct = 0
for wid in est_authors:
     if est_authors[wid] == actual_author[wid]:
            correct += 1
            
print(correct/len(est_authors))

0.1284


## 1-time

In [43]:
# Check perfect BCubed
perfect_cdict = {0: set(['a']),
                 1: set(['b']),
                 2: set(['c'])}
perfect_ldict = perfect_cdict

In [44]:
p = bcubed.precision(perfect_cdict, perfect_ldict)
r = bcubed.recall(perfect_cdict, perfect_ldict)
f1 = bcubed.fscore(p,r)
print(p)
print(r)
print(f1)

1.0
1.0
1.0


In [24]:
# Clean up cluster work_ids
clusters_data['work_id'].head(5)

0    9999   3103826
1    9998   5834200
2    9997   7470267
3    9996   4807733
4    9995   4571358
Name: work_id, dtype: object

In [25]:
clusters_data['str_work_id'] = clusters_data['work_id']

In [26]:
clusters_data['work_id'] = [int(wid) for idx,wid in clusters_data['str_work_id'].str.split()]

In [27]:
clusters_data['work_id']

0        3103826
1        5834200
2        7470267
3        4807733
4        4571358
5       10191302
6       14845740
7       12410085
8        3121919
9         629465
10       2687138
11       2431745
12        176922
13       6075558
14       9872609
15        763988
16       2813195
17        197516
18       4862195
19       5693311
20       8516611
21        745755
22       1158042
23       1030936
24       2272182
25       1036564
26      10809309
27       2423624
28      10494288
29       8169346
          ...   
9970     2239626
9971      241474
9972     1808218
9973     6160036
9974     1713194
9975    10065479
9976     9877172
9977    14308911
9978      667284
9979     7010212
9980     7687483
9981      309545
9982     4533396
9983      495862
9984    13933089
9985    10161755
9986      106619
9987    13451898
9988     9103933
9989      317740
9990     4398161
9991      460585
9992     3071858
9993    11706285
9994    10051442
9995       60671
9996     8764798
9997    117468

In [28]:
# Save to CSV
clusters_data.to_csv('ao3_fanfiction_kmeans_results_10kfics_10kvocab_no-stopwords_yes-names.csv', index=False)